# Atualização de dados de qualidade de energia no estado do Rio de Janeiro

### Favor olhar todo o documento e alterar onde for indicado

In [149]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import ssl
import os 
import pyodbc

### Teste sua conexão com o banco de dados

In [150]:
# Enviando os dados para o SQL

# Com senha
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'SERVIDOR', 
                      database = 'Sandbox_GIN',
                      UID='seu login',
                      PWD='sua senha')

query = '''
    select 
        * 
    from    DEC_FEC_RJ
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

,CONJUNTO,CÓDIGO,N_DE_CONSUMIDORES,DECTOT,FECTOT,DECXP,FECXP,DECXN,FECXN,DECIP,...,FECINO,DECIPC,FECIPC,DECXPC,FECXPC,DECXNC,FECXNC,MES,ANO,COMPANHIA
0,Alcântara,12355,155904,7.26,2.44,0.0,0.0,0.0,0.0,0.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2010,383
1,Araruama,12356,81267,4.03,2.33,0.0,0.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2010,383
2,Areal,12357,15579,7.92,3.15,0.0,0.0,0.0,0.0,0.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2010,383
3,Arraial do Cabo,12358,13302,1.44,1.08,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2010,383
4,Baixada Campista,12359,38167,2.28,1.32,0.0,0.0,0.0,0.0,0.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2010,383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30258,Julius Arp Jr.,15615,32861,0.80,0.55,0.0,0.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,2020,6612
30259,Centenário,15616,8291,1.97,0.66,0.0,0.0,0.0,0.0,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,2020,6612
30260,CONQUISTA,16068,3911,2.92,0.89,0.0,0.0,0.0,0.0,0.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,2020,6612
30261,CONSELHEIRO PAULINO,16069,27696,0.65,0.51,0.0,0.0,0.0,0.0,0.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,2020,6612


In [151]:
# Colocando o link da coleta dos dados
url = 'https://www2.aneel.gov.br/aplicacoes/indicadores_de_qualidade/decFecSegMensal.cfm?mes={}&ano={}&regiao=SE&distribuidora={}&tipo=d'

### Escolha abaixo o ano e mês que deseja realizar a coleta de dados

>Se desejar mais de um mês ou ano coloque os valores entre [] e separados por vírgula.

#### Ex: desejo os 3 ultimos meses de 2012
##### lista_ano = [2012]
##### lista_mes = [10, 11, 12]
>OBS: Não mexer em lista_distribuidores

In [152]:
# Ler com atenção o tópico acima antes de preencher
lista_ano = [2021]
lista_distribuidores = [383,382,5274,6585,6612]
lista_mes = [1,2,3,4,5,6,7,8,9,10,11,12]

In [153]:
# Apagando o arquivo caso ele já exista para não gerar duplicidade
arquivo = 'final_data_1.csv'
if(os.path.exists(arquivo) and os.path.isfile(arquivo)): 
  os.remove(arquivo) 
  print("arquivo deletado") 
else: 
  print("arquivo não encontrado")

arquivo deletado


In [154]:
# Definindo código para coleta de todos os dados escolhidos em cima e criando um csv
for ano in lista_ano:
    for companhia in lista_distribuidores:
        for mes in lista_mes:


            # se conectanto ao site e mudando o padrão decimal para o brasileiro
            scraping = pd.read_html(url.format(mes,ano,companhia), decimal=',', thousands='.')

            # Criando dataframe a partir da tag 'table' no html
            df = pd.DataFrame(scraping[0])

            # Definindo cabeçalho
            df.columns = df.iloc[2]

            #Removendo linhas superiores e inferiores desnecessários
            df.drop(df.tail(3).index,inplace=True)
            df.drop(df.head(3).index,inplace=True)

            # Criando e inserindo dados para mês, ano e companhia (dados inseridos manualmente até aqui)
            df = df.assign(MES = '{}'.format(mes))
            df = df.assign(ANO = '{}'.format(ano))
            df = df.assign(COMPANHIA = '{}'.format(companhia))

            # Juntando a coleta em um só DataFrame
            all_dfs = pd.DataFrame(df)

            #Criando CSV com esses dados
            all_dfs.to_csv('final_data_1.csv', encoding= 'utf-8', mode = 'a', header = False, index=False) 

### Pare de executar o código e vá abrir o CSV no Excel e verificar se foram coletados exatamente os dados que você desejava

In [155]:
# Adicionando cabeçalho ao CSV
df = pd.read_csv('final_data_1.csv', encoding = 'utf-8',
                  names=['CONJUNTO', 'CÓDIGO', 'N_CONSUMIDORES','DECTOT','FECTOT','DECXP','FECXP','DECXN','FECXN','DECIP','FECIP','DECIND','FECIND','DECINE','FECINE','DECINC','FECINC','DECINO','FECINO','DECIPC','FECIPC','DECXPC','FECXPC','DECXNC','FECXNC','MES','ANO','COD_DISTRIBUIDORA'])

In [156]:
# Verificando se os dados estão ok
df

,CONJUNTO,CÓDIGO,N_CONSUMIDORES,DECTOT,FECTOT,DECXP,FECXP,DECXN,FECXN,DECIP,...,FECINO,DECIPC,FECIPC,DECXPC,FECXPC,DECXNC,FECXNC,MES,ANO,COD_DISTRIBUIDORA


> Código utilizado no SQL para criar a tabela
```sql
CREATE TABLE DEC_FEC_RJ(
    CONJUNTO VARCHAR(100),
    CÓDIGO VARCHAR(10),
    N_DE_CONSUMIDORES INT,
    DECTOT FLOAT,
    FECTOT FLOAT,
    DECXP FLOAT,
    FECXP FLOAT,
    DECXN FLOAT,
    FECXN FLOAT,
    DECIP FLOAT,
    FECIP FLOAT,
    DECIND FLOAT,
    FECIND FLOAT,
    DECINE FLOAT,
    FECINE FLOAT,
    DECINC FLOAT,
    FECINC FLOAT,
    DECINO FLOAT,
    FECINO FLOAT,
    DECIPC FLOAT,
    FECIPC FLOAT,
    DECXPC FLOAT,
    FECXPC FLOAT,
    DECXNC FLOAT,
    FECXNC FLOAT,
    MES INT,
    ANO INT,
    COMPANHIA INT
);

```

#### Se conectando ao banco de dados

In [157]:
# Enviando os dados para o SQL

# Com senha
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'SERVIDOR', 
                      database = 'Sandbox_GIN',
                      UID='seu login',
                      PWD='sua senha')
                     
''' 
# Localhost
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Sandbox_Gin')
'''
cursor = conn.cursor()

for index, row in df.iterrows():
    
    cursor.execute('''
    
        INSERT INTO DEC_FEC_RJ (
            CONJUNTO,
            CÓDIGO,
            N_DE_CONSUMIDORES,
            DECTOT,
            FECTOT,
            DECXP,
            FECXP,
            DECXN,
            FECXN,
            DECIP,
            FECIP,
            DECIND,
            FECIND,
            DECINE,
            FECINE,
            DECINC,
            FECINC,
            DECINO,
            FECINO,
            DECIPC,
            FECIPC,
            DECXPC,
            FECXPC,
            DECXNC,
            FECXNC,
            MES,
            ANO,
            COMPANHIA
        ) 
        values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', # mesma quantidade de variáveis

        row['CONJUNTO'],
        row['CÓDIGO'],
        row['N_CONSUMIDORES'],
        row['DECTOT'],
        row['FECTOT'],
        row['DECXP'],
        row['FECXP'],
        row['DECXN'],
        row['FECXN'],
        row['DECIP'],
        row['FECIP'],
        row['DECIND'],
        row['FECIND'],
        row['DECINE'],
        row['FECINE'],
        row['DECINC'],
        row['FECINC'],
        row['DECINO'],
        row['FECINO'],
        row['DECIPC'],
        row['FECIPC'],
        row['DECXPC'],
        row['FECXPC'],
        row['DECXNC'],
        row['FECXNC'],
        row['MES'],
        row['ANO'],
        row['COD_DISTRIBUIDORA']   

    )

conn.commit()
cursor.close()